In [1]:
from datasets import load_dataset
from transformers import BertTokenizer
dataset = load_dataset('dair-ai/emotion')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for dair-ai/emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/dair-ai/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings

In [2]:
dataset


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [3]:
tokenizer

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [4]:
import re
import unidecode
def clean_keywords(keyword):
    cleaned = re.sub(r'%20', ' ', keyword)
    cleaned = unidecode.unidecode(keyword)
    cleaned = re.sub(r"[!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n -' ]"," ",keyword)
    return cleaned

In [5]:
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [6]:
tokenized_datasets["train"]

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 16000
})

In [7]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [22]:
!pip install accelerate -U
!pip install transformers[torch] -U

In [8]:
from transformers import  Trainer, TrainingArguments
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import numpy as np
from sklearn.metrics import accuracy_score

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

model = BertForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=6)

training_args = TrainingArguments(
output_dir="./results",
# Directory for saving outputs
learning_rate=5e-5,
# Learning rate for optimization
per_device_train_batch_size=16,
# Batch size for training
per_device_eval_batch_size=16,
# Batch size for evaluation
num_train_epochs=3,
# Number of training epochs
weight_decay=0.01,
# Weight decay for regularization
evaluation_strategy="epoch",
save_strategy = "no"
# Evaluation is done at the end of each epoch
)
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]
val_dataset = tokenized_datasets["validation"]
trainer = Trainer(
model=model,
args=training_args,
train_dataset=train_dataset,
eval_dataset=eval_dataset,
compute_metrics=lambda p: {"accuracy": accuracy_score(p.label_ids,
np.argmax(p.predictions,
axis=1))}
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [10]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [11]:
eval_results = trainer.evaluate(val_dataset)
print(eval_results)

{'eval_loss': 1.8156042098999023, 'eval_accuracy': 0.09, 'eval_runtime': 67.9655, 'eval_samples_per_second': 29.427, 'eval_steps_per_second': 1.839}


In [13]:
!pip install ray


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 MB 9.8 MB/s eta 0:00:00


In [14]:
from ray import tune
def ray_hp_space(trial):
    return {
        "learning_rate": tune.loguniform(1e-6, 1e-4),
        "per_device_train_batch_size": tune.choice([1,4 ,8]),
        "num_train_epochs": tune.choice([1,3,5]),

        "weight_decay": tune.loguniform(1e-4, 0.1)
    }

In [15]:
small_train_dataset = tokenized_datasets["train"].shard(index=1, num_shards=10)
small_eval_dataset = tokenized_datasets["test"].shard(index=1, num_shards=10)


In [16]:
num_labels = 6
from transformers import  Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import accuracy_score
def model_init():
    return  BertForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=num_labels)

In [17]:




num_labels = num_labels
trainer = Trainer(

    model_init= model_init,

    args=training_args,

    train_dataset=small_train_dataset,

    eval_dataset=small_eval_dataset,

    compute_metrics=lambda p: {"accuracy": accuracy_score(p.label_ids,
np.argmax(p.predictions,
axis=1))}




)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
best_trial = trainer.hyperparameter_search(

    direction="maximize",

    backend="ray",

    hp_space=ray_hp_space,

    n_trials=30

)

2024-03-11 01:05:09,059	INFO worker.py:1724 -- Started a local Ray instance.
2024-03-11 01:05:13,244	INFO tune.py:220 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run(...)`.
2024-03-11 01:05:13,254	INFO tune.py:592 -- [output] This will use the new output engine with verbosity 2. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949
2024-03-11 01:05:13,295	INFO tensorboardx.py:178 -- pip install "ray[tune]" to see TensorBoard files.
2024-03-11 01:05:13,296	WARNING callback.py:137 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`
2024-03-11 01:05:13,300	WARNING tune.py:916 -- AIR_VERBOSITY is set, ignoring p

+-------------------------------------------------------------------+
| Configuration for experiment     _objective_2024-03-11_01-05-13   |
+-------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator            |
| Scheduler                        FIFOScheduler                    |
| Number of trials                 30                               |
+-------------------------------------------------------------------+

View detailed results here: /root/ray_results/_objective_2024-03-11_01-05-13

Trial status: 16 PENDING
Current time: 2024-03-11 01:05:13. Total running time: 0s
Logical resource usage: 0/2 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:T4)
+--------------------------------------------------------------------------------------------------------------------+
| Trial name               status       learning_rate     ..._train_batch_size     num_train_epochs     weight_decay |
+------------------------------------

(_objective pid=3656) 2024-03-11 01:05:34.278362: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(_objective pid=3656) 2024-03-11 01:05:34.278420: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(_objective pid=3656) 2024-03-11 01:05:34.280232: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(_objective pid=3656) 2024-03-11 01:05:35.412830: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
(_objective pid=3656) Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly ini


Trial status: 1 RUNNING | 16 PENDING
Current time: 2024-03-11 01:05:43. Total running time: 30s
Logical resource usage: 1.0/2 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:T4)
+--------------------------------------------------------------------------------------------------------------------+
| Trial name               status       learning_rate     ..._train_batch_size     num_train_epochs     weight_decay |
+--------------------------------------------------------------------------------------------------------------------+
| _objective_6a385_00000   RUNNING        5.61152e-06                        1                    5      0.015703    |
| _objective_6a385_00001   PENDING        1.57513e-05                        8                    3      0.000293754 |
| _objective_6a385_00002   PENDING        1.30667e-06                        1                    5      0.00011528  |
| _objective_6a385_00003   PENDING        8.70602e-05                        4                    3      0.00010

  3%|▎         | 240/8000 [00:35<18:23,  7.03it/s]


...

 33%|███▎      | 200/600 [02:36<05:08,  1.30it/s]
(_objective pid=8866) 
  0%|          | 0/13 [00:00<?, ?it/s]
(_objective pid=8866) 
 15%|█▌        | 2/13 [00:00<00:03,  3.37it/s]
(_objective pid=8866) 
 23%|██▎       | 3/13 [00:01<00:04,  2.39it/s]
(_objective pid=8866) 
 31%|███       | 4/13 [00:01<00:04,  2.07it/s]
(_objective pid=8866) 
 38%|███▊      | 5/13 [00:02<00:04,  1.92it/s]
(_objective pid=8866) 
 46%|████▌     | 6/13 [00:02<00:03,  1.84it/s]
(_objective pid=8866) 
 54%|█████▍    | 7/13 [00:03<00:03,  1.79it/s]
(_objective pid=8866) 
 62%|██████▏   | 8/13 [00:04<00:02,  1.77it/s]
(_objective pid=8866) 
 69%|██████▉   | 9/13 [00:04<00:02,  1.74it/s]
(_objective pid=8866) 
 77%|███████▋  | 10/13 [00:05<00:01,  1.74it/s]
(_objective pid=8866) 
 85%|████████▍ | 11/13 [00:05<00:01,  1.72it/s]
(_objective pid=8866) 
 92%|█████████▏| 12/13 [00:06<00:00,  1.72it/s]



Trial _objective_6a385_00001 finished iteration 1 at 2024-03-11 01:28:32. Total running time: 23min 19s
+-------------------------------------------------+
| Trial _objective_6a385_00001 result             |
+-------------------------------------------------+
| checkpoint_dir_name                             |
| time_this_iter_s                        169.455 |
| time_total_s                            169.455 |
| training_iteration                            1 |
| epoch                                         1 |
| eval_accuracy                             0.615 |
| eval_loss                               1.06991 |
| eval_runtime                             7.3842 |
| eval_samples_per_second                  27.085 |
| eval_steps_per_second                     1.761 |
| objective                                 0.615 |
+-------------------------------------------------+
(_objective pid=8866) {'eval_loss': 1.0699081420898438, 'eval_accuracy': 0.615, 'eval_runtime': 7.3842, 'eval_sampl

(_objective pid=8866) 
                                                 
100%|██████████| 13/13 [00:06<00:00,  2.04it/s]
                                               
 35%|███▌      | 210/600 [02:51<05:37,  1.16it/s]
2024-03-11 01:28:40,908	WARNING tune.py:186 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 



Trial status: 1 TERMINATED | 1 RUNNING | 16 PENDING
Current time: 2024-03-11 01:28:40. Total running time: 23min 27s
Logical resource usage: 1.0/2 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:T4)
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Trial name               status         learning_rate     ..._train_batch_size     num_train_epochs     weight_decay     iter     total time (s)     eval_loss     eval_accuracy     eval_runtime     ...amples_per_second |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| _objective_6a385_00001   RUNNING          1.57513e-05                        8                    3      0.000293754        1         

 37%|███▋      | 223/600 [03:01<04:49,  1.30it/s]
2024-03-11 01:28:50,966	WARNING tune.py:1057 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: tune.run(..., resume=True)
2024-03-11 01:28:51,007	WARNING experiment_analysis.py:193 -- Failed to fetch metrics for 16 trial(s):
- _objective_6a385_00002: FileNotFoundError('Could not fetch metrics for _objective_6a385_00002: both result.json and progress.csv were not found at /root/ray_results/_objective_2024-03-11_01-05-13/_objective_6a385_00002_2_learning_rate=0.0000,num_train_epochs=5,per_device_train_batch_size=1,weight_decay=0.0001_2024-03-11_01-05-13')
- _objective_6a385_00003: FileNotFoundError('Could not fetch metrics for _objective_6a385_00003: both result.json and progress.csv were not found at /root/ray_results/_objective_2024-03-11_01-05-13/_objective_6a385_00003_3_learning_rate=0.0001,num_train_epochs=3,per_device_train_batch_size=4,weight_decay=0.0001_2024-03-11_01-05-13')
- _objec

In [19]:
best_trial

BestRun(run_id='6a385_00000', objective=0.84, hyperparameters={'learning_rate': 5.61151641533451e-06, 'per_device_train_batch_size': 1, 'num_train_epochs': 5, 'weight_decay': 0.015702970884055385}, run_summary=<ray.tune.analysis.experiment_analysis.ExperimentAnalysis object at 0x7d636692a8c0>)

{'learning_rate': 8.70602e-05,
 'per_device_train_batch_size': 4,
 'num_train_epochs': 3,
 'weight_decay': 0.000100539}

In [20]:

train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]
val_dataset = tokenized_datasets["validation"]
trainer = Trainer(

model=model,
args=training_args,
train_dataset=train_dataset,
eval_dataset=eval_dataset,
compute_metrics=lambda p: {"accuracy": accuracy_score(p.label_ids,
np.argmax(p.predictions,
axis=1))}
)

In [21]:
for n, v in best_trial.hyperparameters.items():
    setattr(trainer.args, n, v)



In [22]:
trainer.args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_v2': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_

In [23]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
trainer.save_model('/home/ai/Downloads/results/last-bert-emotion')

In [27]:
predictions = trainer.evaluate(train_dataset)
print(predictions)

Epoch,Training Loss,Validation Loss,Accuracy
0,0.493400,0.324572,0.935000


{'eval_loss': 0.3245723247528076, 'eval_accuracy': 0.935}


In [25]:
predictions = trainer.evaluate(eval_dataset)
print(predictions)

Epoch,Training Loss,Validation Loss,Accuracy
0,0.493400,0.443169,0.918000


{'eval_loss': 0.4431690573692322, 'eval_accuracy': 0.918}


In [26]:
predictions = trainer.evaluate(val_dataset)
print(predictions)

Epoch,Training Loss,Validation Loss,Accuracy
0,0.493400,0.399073,0.922500


{'eval_loss': 0.39907336235046387, 'eval_accuracy': 0.9225}


In [28]:
import random


In [35]:
import random

num_examples = 10
picks = []
lable = []
input = []
prediction = []
for i in range(num_examples):
        pick = random.randint(0, len(dataset['validation'])-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
        input.append(dataset['validation'][picks[i]]['text'])
        #input_token = tokenizer(dataset['validation'][picks[i]]['text'])
        lable.append(dataset['validation'][picks[i]]['label'])
        encoded_input = tokenizer(dataset['validation'][pick]['text'],return_tensors='pt').to(model.device)
        prediction.append(torch.argmax(trainer.model.forward(**encoded_input).logits, dim=1)[0].cpu().item())





In [36]:
print(input)
print(lable,"lable")
print(prediction,"prediction")



['i feel grouchy and i cannot think properly when i am deprived of food for more than two hours', 'i feel not worthwhile', 'i feel unusually mellow not having to worry about any of the aforementioned things not having to rely on tylenol pm or nyquil to lull me to sleep', 'i was questioning myself and feeling nervous about being able to hit the targets', 'i get the feeling he was as surprised as everybody else when people started getting sick', 'i accept the medication until i dont feel too troubled by those i will never have the full benefices from them', 'i get up to refill my coffee and feel that pleasant and familiar ache it reminds me how much i miss the whole body conversations you can have when you re sitting on a good good horse', 'ive explained that he is very creative and loves to makes things and i feel that he is very smart and intelligent and he is lacking in some areas that i agree with', 'i was feeling sentimental', 'when i was walking around all alone at night']
[3, 1, 1